In [1]:
from langchain_community.chat_models.llamacpp import ChatLlamaCpp
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

from langchain.tools import Tool
from langchain.prompts import MessagesPlaceholder

from langchain.chains import LLMChain

import sys
import os
sys.path.append('../../littleSeven/')
from common.config import cfg

In [2]:
from langchain.agents import initialize_agent

In [3]:
from datetime import datetime
from langchain.tools import Tool
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor
from langchain.agents.agent import Agent
from langchain.schema import AgentAction, AgentFinish
from langchain_community.chat_models.llamacpp import ChatLlamaCpp
from typing import List, Tuple, Any
from os.path import expanduser

In [4]:
# prompt_template = PromptTemplate(
#     input_variables=["tools", "history", "agent_scratchpad"],
#     template=(
#         "You are an assistant that can use tools to answer questions.\n"
#         "When deciding to use a tool, respond with:\n"
#         "Action: <tool_name>\n"
#         "Action Input: <tool_input>\n\n"
#         "If you already have the answer, respond with:\n"
#         "Final Answer: <answer>\n\n"
#         "Tools:\n"
#         "{tools}\n\n"
#         "History:\n"
#         "{history}\n\n"
#         "Scratchpad:\n"
#         "{agent_scratchpad}\n"
#     )
# )

In [5]:
# prompt_template = PromptTemplate(
#     input_variables=["tools", "history", "agent_scratchpad", "input"],
#     template=(
#         "You are an assistant that can use tools to answer questions.\n"
#         "When deciding to use a tool, you MUST call the tool and wait for the result before answering.\n"
#         "Respond with:\n"
#         "Action: <tool_name>\n"
#         "Action Input: <tool_input>\n\n"
#         "If you have used a tool and obtained the result, respond with:\n"
#         "Final Answer: <answer>\n\n"
#         "Tools:\n"
#         "{tools}\n\n"
#         "History:\n"
#         "{history}\n\n"
#         "Scratchpad:\n"
#         "{agent_scratchpad}\n\n"
#         "Question:\n"
#         "{input}\n"
#     )
# )

In [6]:
prompt_template = PromptTemplate(
    input_variables=["tools", "history", "agent_scratchpad", "input"],
    template=(
        "You are an assistant that can use tools to answer questions.\n"
        "When deciding to use a tool, you MUST call the tool and wait for the result before answering.\n"
        "Respond with:\n"
        "Action: <tool_name>\n"
        "Action Input: <tool_input>\n\n"
        "If you have used a tool and obtained the result, respond with:\n"
        "Final Answer: <answer>\n\n"
        "Tools:\n"
        "{tools}\n\n"
        "History:\n"
        "{history}\n\n"
        "Scratchpad:\n"
        "{agent_scratchpad}\n\n"
        "Question:\n"
        "{input}\n"
    )
)

In [7]:
# return time
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time. No additional input is needed."
    ),
    Tool(
        name="Calculator",
        func=lambda x: str(eval(x)),
        description="Use this tool for mathematical calculations. Input should be a valid mathematical expression."
    )
]

In [8]:
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish

# class CustomOutputParser(AgentOutputParser):
#     def parse(self, text: str):
#         print(f"Parsing output: {text}")
#         if "Final Answer:" in text:
#             final_answer = text.split("Final Answer:")[1].strip()
#             return AgentFinish({"output": final_answer}, text)
#         elif "Action:" in text and "Action Input:" in text:
#             action = text.split("Action:")[1].split("\n")[0].strip()
#             action_input = text.split("Action Input:")[1].strip()
#             print(f"Parsed Action: {action}, Action Input: {action_input}")
#             return AgentAction(action, action_input, text)
#         else:
#             raise ValueError(f"Could not parse output: {text}")

# class CustomOutputParser(AgentOutputParser):
#     def parse(self, text: str):
#         print(f"Parsing output: {text}")
#         if "Final Answer:" in text:
#             final_answer = text.split("Final Answer:")[1].strip()
#             print(f"Final Answer parsed: {final_answer}")
#             return AgentFinish({"output": final_answer}, text)
#         elif "Action:" in text and "Action Input:" in text:
#             action = text.split("Action:")[1].split("\n")[0].strip()
#             action_input = text.split("Action Input:")[1].strip()
#             print(f"Parsed Action: {action}, Action Input: {action_input}")
#             return AgentAction(action, action_input, text)
#         else:
#             raise ValueError(f"Could not parse output: {text}")

class CustomOutputParser(AgentOutputParser):
    def parse(self, text: str):
        print(f"Parsing output: {text}")
        if "Final Answer:" in text:
            final_answer = text.split("Final Answer:")[1].strip()
            return AgentFinish({"output": final_answer}, text)
        elif "Action:" in text and "Action Input:" in text:
            action = text.split("Action:")[1].split("\n")[0].strip()
            action_input = text.split("Action Input:")[1].strip()
            print(f"Parsed Action: {action}, Action Input: {action_input}")
            return AgentAction(action, action_input, text)
        else:
            raise ValueError(f"Could not parse output: {text}")


In [9]:
from os.path import expanduser

# from langchain_community.llms import LlamaCpp

model_path = expanduser(cfg.llm_gguf_model_path_2)
llm = ChatLlamaCpp(
    model_path=model_path,
    temperature=0.7,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    verbose=True
)

llama_load_model_from_file: using device Metal (Apple M2 Max) - 21845 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../model_gguf/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
l

In [10]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_17589/3307768691.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


In [11]:
# # cunstruct input
# tools_description = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

# print(tools_description)
# # initial invoke，without history and intermediate steps
# inputs = {
#     "tools": tools_description,
#     "history": "",
#     "agent_scratchpad": "",
#     "input": "How are you?"
# }

# # invoke LLMChain
# output = llm_chain.predict(**inputs)


# print("LLM Output:")
# print(output)

In [12]:
from langchain.agents.agent import Agent
from pydantic import BaseModel, Field
from typing import List, Tuple, Any

class CustomAgent(Agent, BaseModel):
    llm_chain: LLMChain = Field(...)
    allowed_tools: List[str] = Field(...)
    output_parser: CustomOutputParser = Field(default_factory=CustomOutputParser)

    @property
    def _agent_type(self) -> str:
        return "custom-agent"

    @property
    def observation_prefix(self) -> str:
        return "Observation:"

    @property
    def llm_prefix(self) -> str:
        return "Thought:"

    def _get_default_output_parser(self):
        return self.output_parser

    @staticmethod
    def create_prompt(tools: List[Tool], **kwargs: Any) -> str:
        tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
        return (
            "You are an assistant that can use the following tools:\n\n"
            f"{tool_descriptions}\n\n"
            "When you decide to use a tool, respond with:\n"
            "Action: <tool_name>\n"
            "Action Input: <tool_input>\n\n"
            "If you already have the answer, respond with:\n"
            "Final Answer: <answer>\n"
        )

    def plan(self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any) -> AgentAction:
        thoughts = "".join(
            [f"{self.llm_prefix} {action.log}\n{self.observation_prefix} {observation}\n" for action, observation in intermediate_steps]
        )
        inputs = {"tools": self.allowed_tools, "history": thoughts, **kwargs}
        full_output = self.llm_chain.predict(**inputs)
        return self.output_parser.parse(full_output)




In [13]:
default_history = ""  # use null if without records
default_agent_scratchpad = ""  # is null initially

In [14]:
# create Agent and Executor
custom_agent = CustomAgent(
    llm_chain=llm_chain,
    allowed_tools=[tool.name for tool in tools]
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=custom_agent,
    tools=tools,
    verbose=True
)

In [15]:
user_input = "What is the local time?"
response = agent_executor.run({
    "tools": "\n".join([f"{tool.name}: {tool.description}" for tool in tools]),
    "history": "",
    "agent_scratchpad": "",
    "input": "What is the local time?"
})
print("Agent Response:", response)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_17589/788152752.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run({




> Entering new AgentExecutor chain...
  Action: LocalTime
Action Input: None

llama_perf_context_print:        load time =    4846.28 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   159 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    10 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5246.78 ms /   169 tokens


Parsing output:   Action: LocalTime
Action Input: None
Parsed Action: LocalTime, Action Input: None
  Action: LocalTime
Action Input: NoneLocalTime tool called, returning: 2025-01-11 19:31:59

Observation:Current local time is: 2025-01-11 19:31:59
Thought:  Action: LocalTime

Llama.generate: 158 prefix-match hit, remaining 1 prompt tokens to eval



Final Answer: The current local time is 10:30 AM.

llama_perf_context_print:        load time =    4846.28 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    23 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     892.76 ms /    24 tokens


Parsing output:   Action: LocalTime
Final Answer: The current local time is 10:30 AM.
  Action: LocalTime
Final Answer: The current local time is 10:30 AM.

> Finished chain.
Agent Response: The current local time is 10:30 AM.
